PHASE 1
Goal: Research-grade transaction timeline

In [1]:
# STEP 1.1 — Dataset Loading

In [2]:
import pandas as pd

# Load both sheets
df_1 = pd.read_excel("/content/online_retail_II.xlsx", sheet_name=0)
df_2 = pd.read_excel("/content/online_retail_II.xlsx", sheet_name=1)

# Combine
df = pd.concat([df_1, df_2], ignore_index=True)

print(df.shape)
df.head()


(1067371, 8)


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [3]:
# STEP 1.2 — Column Sanity Check

In [4]:
df.columns


Index(['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'Price', 'Customer ID', 'Country'],
      dtype='object')

In [5]:
# STEP 1.3 — Hard Cleaning Rules (CLV-Safe)

In [6]:
# Rule 1: Drop missing customers
df = df.dropna(subset=["Customer ID"])


In [8]:
# Rule 2: Convert date properly
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])


In [9]:
# Rule 3: Handle cancellations carefully
df["is_cancelled"] = df["Invoice"].astype(str).str.startswith("C")


In [10]:
# STEP 1.4 — Monetary Value Construction

In [11]:
df["revenue"] = df["Quantity"] * df["Price"]


In [12]:
# STEP 1.5 — Temporal Ordering (MOST IMPORTANT)

In [13]:
df = df.sort_values(
    by=["Customer ID", "InvoiceDate", "Invoice"]
).reset_index(drop=True)


In [14]:
# STEP 1.6 — Build Customer Event Index

In [15]:
df["event_index"] = (
    df.groupby("Customer ID")
      .cumcount()
)


In [16]:
# STEP 1.7 — Phase 1 Validation Checks

In [17]:
# Check monotonic time per customer
check = (
    df.groupby("Customer ID")["InvoiceDate"]
      .apply(lambda x: x.is_monotonic_increasing)
)

print("All customers sorted correctly:", check.all())


All customers sorted correctly: True


In [18]:
# Check customers with at least 2 transactions
(df.groupby("Customer ID").size() >= 2).mean()


np.float64(0.9754291484348704)

In [19]:
# STEP 1.8 — Save Phase 1 Artifact

In [22]:
df["Invoice"] = df["Invoice"].astype(str)
df["StockCode"] = df["StockCode"].astype(str)
df.to_parquet("phase1_clean_transactions.parquet", index=False)